In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import json
import numpy as np
import random
import pandas as pd

from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import ParameterGrid
from google.colab import drive

drive.mount('/content/drive')

# 재현 가능성을 위한 시드 고정
RANDOM_SEED = 18
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Dataloader 시드 고정 (아직 사용 x)
'''
generator = torch.Generator()
generator.manual_seed(RANDOM_SEED)

def worker_init_fn(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

# DataLoader(worker_init_fn=worker_init_fn)
'''

# 데이터 경로 설정
train_csv_path = "/content/drive/MyDrive/Samsung_AI_challenge/Data/train.csv"
test_csv_path = "/content/drive/MyDrive/Samsung_AI_challenge/Data/test.csv"
submission_csv_path = '/content/drive/MyDrive/Samsung_AI_challenge/Data/sample_submission.csv'

# 기타 경로 설정 : 저장되는 데이터 이름 등 (model, method 에 맞게 설정)


Mounted at /content/drive


In [3]:
# MLP - 은닉층 5개
# 하이퍼파라미터 : {"batch_size": 32, "hidden_sizes": [32, 64, 64, 32, 16], "learning_rate": 0.001}
# Epoch : 93
# 제출 성능 : 0.752

class MLP5Hidden(nn.Module):
    def __init__(self, input_size=11, hidden_sizes=[32, 64, 64, 32, 16], output_size=1):
        super(MLP5Hidden, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.fc3 = nn.Linear(hidden_sizes[1], hidden_sizes[2])
        self.fc4 = nn.Linear(hidden_sizes[2], hidden_sizes[3])
        self.fc5 = nn.Linear(hidden_sizes[3], hidden_sizes[4])
        self.fc6 = nn.Linear(hidden_sizes[4], output_size)
        self.prelu = nn.PReLU()

    def forward(self, x):
        x = self.prelu(self.fc1(x))
        x = self.prelu(self.fc2(x))
        x = self.prelu(self.fc3(x))
        x = self.prelu(self.fc4(x))
        x = self.prelu(self.fc5(x))
        x = self.fc6(x)

        return x

In [4]:
# 훈련 데이터를 훈련 + 검증 데이터로 나누고 train_dataset, val_dataset 을 반환하는 함수
def split_dataloader(train_csv_path, val_ratio=0.2, random_state=RANDOM_SEED):
    train_data = pd.read_csv(train_csv_path)

    X = torch.tensor(train_data.iloc[:,1:-1].values, dtype=torch.float32)
    y = torch.tensor(train_data.iloc[:,-1].values, dtype = torch.float32).view(-1,1)

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=val_ratio, random_state=random_state)

    train_dataset = TensorDataset(X_train, y_train)
    val_dataset = TensorDataset(X_val, y_val)

    return train_dataset, val_dataset

train_dataset, val_dataset = split_dataloader(train_csv_path)

In [5]:
# 하이퍼파라미터 범위 설정

param_grid = {
    'hidden_sizes': [[64, 128, 128, 64, 32], [128, 256, 256, 128, 64], [32, 64, 64, 32, 16], [64, 64, 64, 64, 64],
                     [32,32,32,32,32], [64,128,256,126,64], [32, 64, 128, 64, 32], [128,256,512,256,128], [16, 32, 64, 32, 16]],
    'learning_rate': [0.01, 0.001],
    'batch_size': [32, 64, 128],
    }

In [6]:
def train_validation_model(hidden_sizes, learning_rate, batch_size):

    model = MLP5Hidden(hidden_sizes=hidden_sizes).cuda()
    criterion = nn.MSELoss().cuda()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    for epoch in range(100):
        model.train()
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.cuda(), batch_y.cuda()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            batch_X, batch_y = batch_X.cuda(), batch_y.cuda()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            val_loss += loss.item()

    MSE = val_loss / len(val_loader)
    return MSE

In [7]:
def grid_search(parma_grid):
    best_params = None
    best_MSE = float('inf')
    log = open("grid_search_log.txt", "w")
    log.write("Grid Search Log\n")

    for params in ParameterGrid(param_grid):
        MSE = train_validation_model(**params)
        print(f"Testing parameters: {params} => MSE: {MSE:.4f}")
        log.write(f"Testing parameters: {params} => MSE: {MSE:.4f}\n")
        if MSE < best_MSE:
            best_MSE = MSE
            best_params = params
    log.close()
    return best_params, best_MSE


best_params, best_MSE = grid_search(param_grid)
print(f'\nBest parameters: {best_params} with a MSE of {best_MSE:.4f}')

with open(f'MLP5_pReLU_bestparams.json', 'w') as json_file:
    json.dump(best_params, json_file)

Testing parameters: {'batch_size': 32, 'hidden_sizes': [64, 128, 128, 64, 32], 'learning_rate': 0.01} => MSE: 4.6672
Testing parameters: {'batch_size': 32, 'hidden_sizes': [64, 128, 128, 64, 32], 'learning_rate': 0.001} => MSE: 4.1075
Testing parameters: {'batch_size': 32, 'hidden_sizes': [128, 256, 256, 128, 64], 'learning_rate': 0.01} => MSE: 3.7425
Testing parameters: {'batch_size': 32, 'hidden_sizes': [128, 256, 256, 128, 64], 'learning_rate': 0.001} => MSE: 4.1082
Testing parameters: {'batch_size': 32, 'hidden_sizes': [32, 64, 64, 32, 16], 'learning_rate': 0.01} => MSE: 3.5100
Testing parameters: {'batch_size': 32, 'hidden_sizes': [32, 64, 64, 32, 16], 'learning_rate': 0.001} => MSE: 3.2709
Testing parameters: {'batch_size': 32, 'hidden_sizes': [64, 64, 64, 64, 64], 'learning_rate': 0.01} => MSE: 3.1984
Testing parameters: {'batch_size': 32, 'hidden_sizes': [64, 64, 64, 64, 64], 'learning_rate': 0.001} => MSE: 3.4367
Testing parameters: {'batch_size': 32, 'hidden_sizes': [32, 32, 

In [ ]:
# 텍스트를 파일로 저장하는 코드
text_to_save = ""

# 파일 경로와 이름 지정
output_file_path = "/path/to/output_file.txt"  # 경로와 파일명을 원하는 대로 지정하세요

# 파일에 텍스트 저장
with open(output_file_path, "w") as file:
    file.write(text_to_save)

print(f"텍스트가 {output_file_path}에 저장되었습니다.")

In [ ]:
'''
MLP5Hidden model Grid Search log

Testing parameters: {'batch_size': 32, 'hidden_sizes': [64, 128, 128, 64, 32], 'learning_rate': 0.01} => MSE: 4.6672
Testing parameters: {'batch_size': 32, 'hidden_sizes': [64, 128, 128, 64, 32], 'learning_rate': 0.001} => MSE: 4.1075
Testing parameters: {'batch_size': 32, 'hidden_sizes': [128, 256, 256, 128, 64], 'learning_rate': 0.01} => MSE: 3.7425
Testing parameters: {'batch_size': 32, 'hidden_sizes': [128, 256, 256, 128, 64], 'learning_rate': 0.001} => MSE: 4.1082
Testing parameters: {'batch_size': 32, 'hidden_sizes': [32, 64, 64, 32, 16], 'learning_rate': 0.01} => MSE: 3.5100
Testing parameters: {'batch_size': 32, 'hidden_sizes': [32, 64, 64, 32, 16], 'learning_rate': 0.001} => MSE: 3.2709
Testing parameters: {'batch_size': 32, 'hidden_sizes': [64, 64, 64, 64, 64], 'learning_rate': 0.01} => MSE: 3.1984
Testing parameters: {'batch_size': 32, 'hidden_sizes': [64, 64, 64, 64, 64], 'learning_rate': 0.001} => MSE: 3.4367
Testing parameters: {'batch_size': 32, 'hidden_sizes': [32, 32, 32, 32, 32], 'learning_rate': 0.01} => MSE: 3.9137
Testing parameters: {'batch_size': 32, 'hidden_sizes': [32, 32, 32, 32, 32], 'learning_rate': 0.001} => MSE: 3.3493
Testing parameters: {'batch_size': 32, 'hidden_sizes': [64, 128, 256, 126, 64], 'learning_rate': 0.01} => MSE: 6.5530
Testing parameters: {'batch_size': 32, 'hidden_sizes': [64, 128, 256, 126, 64], 'learning_rate': 0.001} => MSE: 3.4354
Testing parameters: {'batch_size': 32, 'hidden_sizes': [32, 64, 128, 64, 32], 'learning_rate': 0.01} => MSE: 3.7004
Testing parameters: {'batch_size': 32, 'hidden_sizes': [32, 64, 128, 64, 32], 'learning_rate': 0.001} => MSE: 4.0292
Testing parameters: {'batch_size': 32, 'hidden_sizes': [128, 256, 512, 256, 128], 'learning_rate': 0.01} => MSE: 22.7032
Testing parameters: {'batch_size': 32, 'hidden_sizes': [128, 256, 512, 256, 128], 'learning_rate': 0.001} => MSE: 3.7145
Testing parameters: {'batch_size': 32, 'hidden_sizes': [16, 32, 64, 32, 16], 'learning_rate': 0.01} => MSE: 3.2547
Testing parameters: {'batch_size': 32, 'hidden_sizes': [16, 32, 64, 32, 16], 'learning_rate': 0.001} => MSE: 3.2312
Testing parameters: {'batch_size': 64, 'hidden_sizes': [64, 128, 128, 64, 32], 'learning_rate': 0.01} => MSE: 3.2315
Testing parameters: {'batch_size': 64, 'hidden_sizes': [64, 128, 128, 64, 32], 'learning_rate': 0.001} => MSE: 4.4115
Testing parameters: {'batch_size': 64, 'hidden_sizes': [128, 256, 256, 128, 64], 'learning_rate': 0.01} => MSE: 7.5632
Testing parameters: {'batch_size': 64, 'hidden_sizes': [128, 256, 256, 128, 64], 'learning_rate': 0.001} => MSE: 4.4371
Testing parameters: {'batch_size': 64, 'hidden_sizes': [32, 64, 64, 32, 16], 'learning_rate': 0.01} => MSE: 3.3326
Testing parameters: {'batch_size': 64, 'hidden_sizes': [32, 64, 64, 32, 16], 'learning_rate': 0.001} => MSE: 3.2585
Testing parameters: {'batch_size': 64, 'hidden_sizes': [64, 64, 64, 64, 64], 'learning_rate': 0.01} => MSE: 3.2405
Testing parameters: {'batch_size': 64, 'hidden_sizes': [64, 64, 64, 64, 64], 'learning_rate': 0.001} => MSE: 3.9393
Testing parameters: {'batch_size': 64, 'hidden_sizes': [32, 32, 32, 32, 32], 'learning_rate': 0.01} => MSE: 3.2616
Testing parameters: {'batch_size': 64, 'hidden_sizes': [32, 32, 32, 32, 32], 'learning_rate': 0.001} => MSE: 3.3527
Testing parameters: {'batch_size': 64, 'hidden_sizes': [64, 128, 256, 126, 64], 'learning_rate': 0.01} => MSE: 3.3493
Testing parameters: {'batch_size': 64, 'hidden_sizes': [64, 128, 256, 126, 64], 'learning_rate': 0.001} => MSE: 3.2871
Testing parameters: {'batch_size': 64, 'hidden_sizes': [32, 64, 128, 64, 32], 'learning_rate': 0.01} => MSE: 4.4585
Testing parameters: {'batch_size': 64, 'hidden_sizes': [32, 64, 128, 64, 32], 'learning_rate': 0.001} => MSE: 4.7177
Testing parameters: {'batch_size': 64, 'hidden_sizes': [128, 256, 512, 256, 128], 'learning_rate': 0.01} => MSE: 55.1814
Testing parameters: {'batch_size': 64, 'hidden_sizes': [128, 256, 512, 256, 128], 'learning_rate': 0.001} => MSE: 3.2354
Testing parameters: {'batch_size': 64, 'hidden_sizes': [16, 32, 64, 32, 16], 'learning_rate': 0.01} => MSE: 3.7065
Testing parameters: {'batch_size': 64, 'hidden_sizes': [16, 32, 64, 32, 16], 'learning_rate': 0.001} => MSE: 3.5702
Testing parameters: {'batch_size': 128, 'hidden_sizes': [64, 128, 128, 64, 32], 'learning_rate': 0.01} => MSE: 3.4221
Testing parameters: {'batch_size': 128, 'hidden_sizes': [64, 128, 128, 64, 32], 'learning_rate': 0.001} => MSE: 3.2562
Testing parameters: {'batch_size': 128, 'hidden_sizes': [128, 256, 256, 128, 64], 'learning_rate': 0.01} => MSE: 24.2949
Testing parameters: {'batch_size': 128, 'hidden_sizes': [128, 256, 256, 128, 64], 'learning_rate': 0.001} => MSE: 3.2622
Testing parameters: {'batch_size': 128, 'hidden_sizes': [32, 64, 64, 32, 16], 'learning_rate': 0.01} => MSE: 3.6337
Testing parameters: {'batch_size': 128, 'hidden_sizes': [32, 64, 64, 32, 16], 'learning_rate': 0.001} => MSE: 3.2692
Testing parameters: {'batch_size': 128, 'hidden_sizes': [64, 64, 64, 64, 64], 'learning_rate': 0.01} => MSE: 3.7084
Testing parameters: {'batch_size': 128, 'hidden_sizes': [64, 64, 64, 64, 64], 'learning_rate': 0.001} => MSE: 3.2868
Testing parameters: {'batch_size': 128, 'hidden_sizes': [32, 32, 32, 32, 32], 'learning_rate': 0.01} => MSE: 3.7694
Testing parameters: {'batch_size': 128, 'hidden_sizes': [32, 32, 32, 32, 32], 'learning_rate': 0.001} => MSE: 3.5605
Testing parameters: {'batch_size': 128, 'hidden_sizes': [64, 128, 256, 126, 64], 'learning_rate': 0.01} => MSE: 3.8449
Testing parameters: {'batch_size': 128, 'hidden_sizes': [64, 128, 256, 126, 64], 'learning_rate': 0.001} => MSE: 3.2439
Testing parameters: {'batch_size': 128, 'hidden_sizes': [32, 64, 128, 64, 32], 'learning_rate': 0.01} => MSE: 3.4654
Testing parameters: {'batch_size': 128, 'hidden_sizes': [32, 64, 128, 64, 32], 'learning_rate': 0.001} => MSE: 3.2723
Testing parameters: {'batch_size': 128, 'hidden_sizes': [128, 256, 512, 256, 128], 'learning_rate': 0.01} => MSE: 3.9837
Testing parameters: {'batch_size': 128, 'hidden_sizes': [128, 256, 512, 256, 128], 'learning_rate': 0.001} => MSE: 3.4511
Testing parameters: {'batch_size': 128, 'hidden_sizes': [16, 32, 64, 32, 16], 'learning_rate': 0.01} => MSE: 4.0902
Testing parameters: {'batch_size': 128, 'hidden_sizes': [16, 32, 64, 32, 16], 'learning_rate': 0.001} => MSE: 3.7863

Best parameters: {'batch_size': 32, 'hidden_sizes': [64, 64, 64, 64, 64], 'learning_rate': 0.01} with a MSE of 3.1984
'''